 # Variational quantum circuits tutorial

  In this tutorial you will create your own variational quantum eigensolver (VQE) algorithm. You will also see how to apply it to find the ground state energy of the transverse field Ising model. All of the python libraries you will need for this tutorial are listed bellow.

In [ ]:
import numpy as np
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
from collections import namedtuple
from functools import reduce

 ### Bring Your Own Simulator

In [ ]:
# Configuration
Instruction = namedtuple("Instruction", "gate bits symbols")
ket0, ket1 = np.array([1, 0]), np.array([0, 1])


def simulate_shots(psi, n_shots):
    # We will come back to this later.
    pass


def variational_quantum_simulator(program, nq, initial_qubits=None, symbols=None):
    if initial_qubits is None:
        initial_qubits = [ket0] * nq
    state = initial_qubits[0]
    for qubit in initial_qubits[1:]:
        state = np.array(np.kron(state, qubit))
    state = state.reshape([2 for i in range(nq)])
    for instruction in program:
        u = instruction.gate
        bits = instruction.bits
        instruction_symbols = instruction.symbols
        if len(u.shape) == 2:
            # if u is provided as a matrix, we turn it into the correct tensor
            n_inputs = len(bits)
            idx_range = int(
                u.shape[0] ** (1 / n_inputs)
            )  # matrix is square so we can take first or second axis
            if isinstance(u, sp.Matrix):
                # convert to numpy matrix
                f = sp.lambdify(list(u.free_symbols), u, "numpy")
                u = f(*[symbols[symbol] for symbol in instruction_symbols])
            u = u.reshape([idx_range for i in range(n_inputs * 2)])
        state = np.tensordot(state, u, axes=[bits, [i for i in range(len(bits))]])
        # tensordot reorders the tensor to uncontracted+contracted and we want the original, so we undo their reording:
        current_order = [bit for bit in range(nq) if bit not in bits] + bits
        state = state.transpose([current_order.index(i) for i in range(nq)])
    return state



 ### Gate primitives
 Let's create primitive operations for our quantum computer, first we'll create non-parametric gates.

 Give the definition of Identity, PauliX, PauliY, PauliZ, Hadamard and CNOT gates as numpy arrays (by replacing the None values below).

In [ ]:
# Operations
I = None
X = None
Y = None
Z = None
H = None
CN = None


Our simulator handles parameterised gates as sympy matrices and explicit gates, like the ones above, as numpy arrays.

Now for the parametric gates, we'll create a general rotation gate as a function that takes in a Pauli matrix (X,Y or Z) and returns the exponential of that pauli, i.e. $e^{-i\theta\sigma_k}, \forall  k \in \{x,y,z\}$. Note that *inner angles* should have a factor of two. Calling `Rs` with Pauli `X`, for example, should yield the following output:
 ```python
 Rs(X)
 Matrix([
 [   cos(θ/2), -I*sin(θ/2)],
 [-I*sin(θ/2),    cos(θ/2)]])
 ```
 Note that a sp.Matrix object is returned with a symbol called θ

In [ ]:
x0 = sp.symbols("θ")  # The unicode for θ us u03B8
Rs = None

 ### Quantum programming
 Now we can write some quantum programs. Whether quantum or classical, a program is just a list of instructions. Our simulator expects just that, we define a program as a python list where the elements are Instructions. We've provided an `Instruction` class using python's namedtuple at the top of the file (it's just a fancy list). You can instantiate single instructions by:
 - providing the gate as a numpy (if explicit) or sympy (if parametric) matrix,
 - a list containing the qubit indices that the gate acts on
 - and individual symbols, if it is a parametric gate.

 For example, applying hadamard on qubits 2 and 3 (using indices starting at 0):
 ```python
 instruction1 = Instruction(H, [2], None)
 instruction2 = Instruction(H, [3], None)
 ```
 Two qubit gates, like $CN_{70}$ (a cnot with qubit 7 as control and qubit 0 as target)
 looks like:
 ```python
 instruction3 = Instruction(CN, [7, 0], None)
 ```
 An example of a parameterised instruction is applying an Rx gate on the qubit at index 4
 ```python
 x0 = sp.symbols("x0")
 instruction4 = Instruction(Rs(X), [4], [x0])
 ```

 Finally, if you want to create a program that runs the above instructions on the initial state is $|{00\cdots0}\rangle$, and gets the final statevector, you can do the following:
 ```python
 program = [instruction1, instruction2, instruction3, instruction4]
 nq = 8
 initial_qubits = [ket0] * nq
 psi = variational_quantum_simulator(program, nq, initial_qubits)
 ```
 Study the two programs below and see if their outputs make sense to you.

In [ ]:
instruction1 = Instruction(X, [0], None)
instruction2 = Instruction(X, [3], None)
instruction3 = Instruction(CN, [3, 0], None)
program1 = [instruction1, instruction2]
program2 = [instruction1, instruction2, instruction3]


nq = 5
initial_qubits = [ket0] * nq
# Do nothing
psi = variational_quantum_simulator([], nq, initial_qubits)
print(np.where(psi == 1))

nq = 5
initial_qubits = [ket0] * nq
# Run program1
psi = variational_quantum_simulator(program1, nq, initial_qubits)
print(np.where(psi == 1))

nq = 5
initial_qubits = [ket0] * nq
# Run program2
psi = variational_quantum_simulator(program2, nq, initial_qubits)
print(np.where(psi == 1))

 ## Variational Quantum Eigensolver

 We are now ready to apply the Variational Quantum Eigensolver (VQE) to obtain the ground state energy of a Hamiltonian $H$, that is an eigenstate of $H$ corresponding to minimal energy. In this tutorial we will apply VQE to the transverse field Ising model. VQE is a hybrid quantum-classical algorithm where a quantum computer aids in the calculation of expectation values of observables. The quantum computer is used to prepare a trial state $|{\psi(\theta)}\rangle$ where $\theta$ is determined classically according to the goal of minimizing the expectation value of the energy with respect to $|{\psi(\theta)}\rangle$, $E = \langle{\psi(\theta)}|H|{\psi(\theta)}\rangle$. Finding the parameter(s) $\theta$ for which $E$ is minimum allows us to estimate the groundstate $|{g}\rangle$ of $H$, i.e. where $H|{g}\rangle=E_g|{g}\rangle$ and where any state $|{\psi(\theta)}\rangle$ parameterised by $\theta$ satisfies $\langle{\psi(\theta)}|H|{\psi(\theta)}\rangle\geq E_g$. This is called the variational principle.

 ### The transverse field Ising model
 The model we consider is a one dimensional Ising model with a transverse field on a chain lattice with periodic boundary conditions:
 $$H=-\frac{J}{4}\sum_{\langle ij \rangle} \sigma_i^z \sigma_j^z -\frac{h}{2}\sum_i \sigma_i^x \,,$$
  where $\langle ij \rangle$ refers to nearest neighbours, $J$ is the coupling strength and $h$ is the strength of the transverse field. We will consider the case where $J=1$ (our units of energy) and $h\in[0,2.2]$. To compute $\langle{\psi}| H |{\psi}\rangle$ we make use of linearity, and split up the computation as follows: $$\langle{\psi|H|\psi}\rangle=-\frac{J}{4}\sum_{\langle ij \rangle}\langle{\psi|\sigma^z_i\sigma^z_j|\psi}\rangle-\frac{h}{2}\sum_i\langle{\psi|\sigma^x_i|\psi}\rangle$$ Our prescription to apply VQE is then as follows:
 - Pick a circuit ansatz $U(\theta)$ s.t. $U(\theta)|{00\cdots0}\rangle=|{\psi(\theta)}\rangle$.
 - Execute the circuit and use the final state vector to simulate measurements in the computational basis, i.e. compute the frequency distribution for the state $|{\psi(\theta)}\rangle$ for $N$ repetitions of the experiment.
   - Use the frequency distribution to calculate $\langle{\psi|\sigma^z_i\sigma^z_j|\psi}\rangle$. For example, consider a two qubit state where $\sigma^z\otimes\sigma^z |{ij}\rangle = (-1)^{i+j}|{ij}\rangle$ so that $$\langle{\psi|\sigma^z_i\sigma^z_j|\psi}\rangle = \frac{1}{N}\sum_{i,j=0}^1 (-1)^{i+j}f_{ij}|{ij}\rangle\,.$$
 - Execute the same circuit but with a cycle of hadamards applied to each qubit before measurement, to simulate a measurement in the X basis.
   - Use the frequency distribution to calculate $\langle{\psi|\sigma^x_i|\psi}\rangle$.
 - Multiply and add the terms with the correct constants to get $\langle{\psi|H|\psi}\rangle$.
 - Update $\theta$ and repeat until $\langle{\psi|H|\psi}\rangle$ is minimised.

 ### Circuit ansatz
 We now make an educated guess for the structure of $U(\theta)$ and then implement it with our simulator. Our guess is based on the observation that the Hamiltonian is invariant under a global bit flip (spin flip). Consider the unitary $U=\prod_i \sigma^x_i$ that flips all the qubits simultaneously. We have that:
 - $U^\dagger\sigma^z_jU=\sigma^x_j \sigma^z_j \sigma^x_j =i\sigma^x_j\sigma^y_j=ii\sigma^z_j=-\sigma^z_j$
    - There's two $\sigma^z$ terms in the Ising part and so the minus cancels in each case.
 - $U^\dagger\sigma^x_jU=\sigma^x_j \sigma^x_j \sigma^x_j =\sigma^x_j$

 Therefore $U^\dagger HU=H\implies HU=UH$, meaning they commute.

Now consider a groundstate $|{g}\rangle$ of the Hamiltonian, $H|{g}\rangle=E|{g}\rangle$, and note that
$$H(U|{g}\rangle) = UH|{g}\rangle=E_g(U|{g}\rangle)\,,$$
so that $U|{g}\rangle$ is also a groundstate of $H$ with the same eigenvalue $E_g$ (i.e. the groundstate is degenerate, which leads to interesting things like spontaneous symmetry breaking). When the driving field is switched off, i.e. $h=0$, both $|{00\cdots0}\rangle$ and $|{11\cdots1}\rangle$ are valid groundstates of the transverse field Ising model (we take this as given and leave the proof as an exercise to the reader). We will use a superposition of these two states $\alpha|{00\cdots0}\rangle+\beta|{11\cdots1}\rangle$ as a starting point for our ansatz. While it is not a groundstate of $H$ for $h\neq0$, we can assume that it is closer to the groundstate than some completely randomly initialized state.

Below we describe how we will compute the circuits need to compute the expectation values of the Hamiltonian with respect to $|{\psi(\theta)}\rangle$.

 The `program_zz` (Ising part - pairwise interactions):
- Initial state is $|{00\cdots0}\rangle$.
- Apply a general rotation on the first qubit to obtain $\alpha|{00\cdots0}\rangle+\beta|{100\cdots 0}\rangle$.
   - A example of a general rotation on the first qubit is $Rz(x_0)Ry(x_1)|{00\cdots0}\rangle=\cos(\frac{x_1}{2})|{00\cdots0}\rangle+\sin(\frac{x_1}{2})e^{ix_0}|{100\cdots0}\rangle$.
- Apply a cycle of cnots to obtain $\alpha|{00\cdots0}\rangle+\beta|{11\cdots1}\rangle$.
   - This is given by $\text{CN}_{(n-1)n}\cdots\text{CN}_{12}\text{CN}_{01}|{\psi}\rangle$, where $\text{CN}_{ij}$ means the $i$ th qubit is the control and the $j$ th qubit is the target.
- Apply $Ry(x_2)$ on each qubit.
   - This will increase the expressivity of our ansatz.


 The `program_x` (transverse field):
- Do `program_zz`
- Apply one a cycle of hadamards (one hadamard gate applied on each qubit) to simulate a measurement in the X basis.


In [ ]:
# use these symbols
# symbol_list = sp.symbols("x0:3")
x0 = sp.symbols("x0")
x1 = sp.symbols("x1")
x2 = sp.symbols("x2")
symbol_list = [x0,x1,x2]

# Specify number of qubits
nq = 4

program_zz = None
program_x = None

### Objective function
Now it's time to compute the expectation value of our Hamiltonian with respect to $|{\psi}\rangle$. We do this in a function that receives a particular set of `symbols_values` (as will be determined by the optimizer) for $|{\psi}\rangle$. Our first job is to emulate the experimental set-up. Notice our circuit simulator returns a statevector. In an experiment this would not be the case, we would instead have a set of measurement outcomes (1 and -1 values that allow us to distinguish the states $|{0}\rangle, |{1}\rangle$) for each qubit, with the size of this set depending on the number of times the experiment was repeated.

Create the function `simulate_shots` (top of page) that takes in a statevector `psi` and the number of shots (experiments) and returns a dictionary with the measured state as keys and the measurement counts as the values. Ensure that only positive measurement counts are returned.


For example, the state $|{\psi}\rangle=\frac{1}{\sqrt{3}}|{000}\rangle+\frac{1}{\sqrt{6}}|{011}\rangle-\frac{1}{\sqrt{2}}|{101}\rangle$ should yield the following dictionary:
```python
simulate_shots(psi,1000)
{(0, 0, 0): 333.0, (0, 1, 1): 167.0, (1, 0, 1): 500.0}
```

Next use the outcomes `get_shots_zz` and `get_shots_x`, shown bellow, to calculate $\sum_{\langle{ij}\rangle}\langle{\sigma^z_i\sigma^z_j}\rangle \text{ and } \sum_{i} \langle{\sigma^x_i}\rangle$.

 **Remember to multiply them with the correct constants**, i.e. measurement outcome, see section *the transverse field Ising model*. Also remember that we are considering **periodic boundary conditions** for our 1d chan, therefore the $n$ th and first qubit are connected.

In [ ]:
def objective_function(symbols_vals, h, J=1, n_shots=1000):
    symbols = {symbol_list[i]: symbols_vals[i] for i in range(len(symbols_vals))}
    psi_zz = variational_quantum_simulator(program_zz, nq, symbols=symbols)
    psi_x = variational_quantum_simulator(program_x, nq, symbols=symbols)
    get_shots_zz = simulate_shots(psi_zz, n_shots)
    get_shots_x = simulate_shots(psi_x, n_shots)

    # calculate expectation values
    exp_zz = None
    exp_x = None
    ising_e = J * (-1 / 4) * exp_zz
    tansverse_e = h * (-1 / 2) * exp_x

    return ising_e + tansverse_e



 ### Optimisation
 Below is an example of the optimisation loop, we use scipy's COBYLA optimizer and enumerate possible $h$ values ($J$ is fixed at one since only the relative magnitudes of $h$ and $J$ is important). A potential issue with this method is that we might get stuck in a local minima. Update the code below by adding an inner loop that runs the optimizer 10 times using different `symbol_vals` and picks the best result for each value $h$. This best result can then be appended to the `results_site` list.

 We are interrested in the average energy per site (we'll compare it to the exact solution), so we divide the resulting energy by the number of qubits: `result.fun/ nq`

In [ ]:
h_vals = np.arange(0.01, 2.2, 0.2)
results_site = []
for h in h_vals:
    symbol_vals = 2 * np.pi * np.random.uniform(size=3)
    result = sc.optimize.minimize(
        objective_function,
        symbol_vals,
        args=(h),
        method="COBYLA",
        options={"maxiter": 20},
    )
    # result.fun is the energy value and we consider average energy per site
    results_site.append(result.fun/ nq)
    print(result.fun / nq)


 ### Visualisation
 Finally let's visualise our results. Bellow we plot the energy per site vs the magnetic field strength. We also plot the exact solution (numerically integrated) for comparison.

In [ ]:
plt.plot(h_vals, results_site, "m--o")
plt.xlabel("transverse field $B [J]$")
plt.ylabel("groundstate energy per site $E_{0}/N [J]$")
plt.show()

def num_integrate_gs(B):
    """
    numerically integrate exact band to get gs energy of TIM
    this should give -E_0/(N*J) by Pfeufy
    Here set J=1 (units of energy)
    """
    # lamba_ratio (setting J=1): compare thesis
    ll = 1 / (2 * B)

    # set energy
    gs_energy = 0

    # numerical integration
    step_size = 0.0001
    k_values = np.arange(0, np.pi, step_size)
    integration_values = [
        step_size * np.sqrt(1 + ll**2 + 2 * ll * np.cos(kk)) for kk in k_values
    ]
    integral = np.sum(integration_values)
    gs_energy = 1 * integral / (4 * np.pi * ll)

    return gs_energy


# plot exact gs energy of TIM vs VQE results
x = np.arange(0.01, 2.2, 0.2)
y = [-1 * num_integrate_gs(xx) for xx in x]
# plot exact results
plt.plot(x, y, "b--s", label="exact")
# plot vqe results
plt.plot(h_vals, results_site, "m--o", label="VQE")
plt.xlabel("magnetic field $B [J]$")
plt.ylabel("groundstate energy $E_{0}/N [J]$")
plt.tight_layout()
plt.legend()